<h1><center>CSCE-5222 Feature Engineering</h1>

<h2><center>ICE-1 </h2>

<h1>Imports</h1>

In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np
import requests

<h3>Download punkt</h3>

In [2]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

------------------------------------------------------------


[nltk_data] Downloading package punkt to C:\Users\RAJESH
[nltk_data]     NEMANI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\RAJESH
[nltk_data]     NEMANI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

<h3> StopWords</h3>

In [3]:
# Downloading the stop words list
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\RAJESH
[nltk_data]     NEMANI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
# Article Link
#link = "https://www.bbc.com/news/business-58390290"
link = "https://www.sehinc.com/news/7-articles-about-autonomous-vehicles-every-official-should-read"

<h3>WebScraping</h3>

In [30]:
page = requests.get(link)

In [31]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content,'html.parser') #parsing the page content

In [32]:
#verification of contents in soup
soup

<!DOCTYPE html>

<html dir="ltr" lang="en" prefix="og: http://ogp.me/ns# content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema#">
<head>
<meta charset="utf-8"/>
<link href="https://www.sehinc.com/sites/default/files/seh__favicon.gif" rel="shortcut icon" type="image/gif"/>
<meta content="There are over three million articles about autonomous vehicles. Here are seven worthy of every official’s attention. The future is getting closer to the present every day as vehicle manufacturers and tech companies race to deliver autonomous vehicles to the masses. Some view this Jetsons-like future as the next wave of human advancement, while others are more skeptical." name="description"/>
<meta content="Drupal 7 (https://www.drupal.org)" name="generator"

In [33]:
contents  = soup.find_all('p')
len(contents)

30

In [34]:
news_contents = []
list_paragraphs = []
for p in np.arange(0,len(contents)):
    paragraph = contents[p].get_text()
    list_paragraphs.append(paragraph)
    final_article = "".join(list_paragraphs)
news_contents.append(final_article)


In [35]:
News_contents= news_contents[0]
News_contents

'\nJump to navigation\n\n\t\tThere are over three million articles about autonomous vehicles. Here are seven worthy of every official’s attention. \xa0\n\t\tThe future is getting closer to the present every day as vehicle manufacturers and tech companies race to deliver autonomous vehicles to the masses. Some view this Jetsons-like future as the next wave of human advancement, while others are more skeptical. Whatever your viewpoint, this emerging technology will impact our cities.\n\t\tMany are writing about autonomous vehicles, but what articles should we be paying attention to? We put together a list of seven articles that give you a solid understanding of the state of autonomous vehicles today.\nINFOGRAPHIC: Everything you need to know about autonomous vehicles\n\t\tBusiness Insider | Jeff Desjardins\n\t\tFor the uninitiated, this article, which includes an in-depth infographic, gives a good foundation by explaining the background, history and mechanics of autonomous vehicles. You 

<h3>Text cleaning and preparation</h3>

In [49]:
#replacing "'s'","\r","\n"
# " when quoting text

News_contents.replace("'s", "")
df = pd.DataFrame({News_contents})
df['Content'] = News_contents
df['Content_Parsed_1'] =(((((News_contents.replace("\r", " ")).replace("\n", " ")).replace("    ", " ")).replace('"', '')).replace("\t", " "))
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

In [50]:
# Removing punctuation
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

<ipython-input-50-161e18e11517>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')


In [51]:
# Removing possessive pronouns
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

<h3>Stemming and Lemmatization</h3>

In [52]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

In [53]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [54]:
df['Content_Parsed_5'] = lemmatized_text_list

In [55]:
# Loading the stop words in english
stop_words = list(stopwords.words('english'))

In [56]:
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [57]:
example = "me eating a meal"
word = "me"

# The regular expression is:
regex = r"\b" + word + r"\b"  # we need to build it like that to work properly

re.sub(regex, "StopWord", example)

'StopWord eating a meal'

In [58]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

<ipython-input-58-3b7196a1b53b>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')


In [59]:
df.head(1)

,0,Content,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,\nJump to navigation\n\n\t\tThere are over thr...,\nJump to navigation\n\n\t\tThere are over thr...,Jump to navigation There are over three mi...,jump to navigation there are over three mi...,jump to navigation there are over three mi...,jump to navigation there are over three mi...,jump to navigation there be over three mil...,jump navigation three million article ...


In [60]:
list_columns = ["Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

In [61]:
X_new= df['Content_Parsed']

In [62]:

loaded_model= pickle.load(open('C:\\Users\\RAJESH NEMANI\\OneDrive\\Documents\\FeatureEngineering\\ICE_1\\0. Latest News Classifier\\04. Model Training\\Models\\best_mnbc.pickle','rb'))
X_train =pickle.load(open('C:\\Users\\RAJESH NEMANI\\OneDrive\\Documents\\FeatureEngineering\\ICE_1\\0. Latest News Classifier\\03. Feature Engineering\\Pickles/X_train.pickle','rb'))
y_train =pickle.load(open('C:\\Users\\RAJESH NEMANI\\OneDrive\\Documents\\FeatureEngineering\\ICE_1\\0. Latest News Classifier\\03. Feature Engineering\\Pickles/y_train.pickle','rb'))


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.19.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [63]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [64]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_new = tfidf.transform(X_new).toarray()
print(features_new.shape)

(1891, 300)
(1, 300)


In [65]:
features_new

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.14730914, 0.        , 0.        , 0.        , 0.        ,
        0.06817927, 0.03816566, 0.07604947, 0.        , 0.07078869,
        0.07202419, 0.12804639, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.0669721 ,
        0.        , 0.        , 0.        , 0.1268215 , 0.07285275,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.07449884, 0.        , 0.        , 0.        , 0.13627142,
        0.07648015, 0.        , 0.        , 0.        , 0.0789949 ,
        0.        , 0.06825673, 0.08100137, 0.06944917, 0.        ,
        0.        , 0.        , 0.        , 0.10273025, 0.10136469,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.09916203, 0.        , 0.        , 0.        , 0.07680821,
        0.        , 0.2316774 , 0.07139863, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.07

In [67]:
loaded_model.predict(features_new)

array([4], dtype=int64)